<a href="https://colab.research.google.com/github/Pradxpk-88/RAG/blob/main/RAG_Phase_2_03_embeddings_and_vector_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Drive & Setup**

In [32]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Install dependencies (CPU only)
!pip install sentence-transformers faiss-cpu

**Imports & Load Semantic Chunks**

In [34]:
# Imports
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [35]:
# Load chunks from Drive

CHUNKS_PATH = "/content/drive/MyDrive/RAG/semantic_chunks.json"

with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Total chunks loaded: {len(chunks)}")


Total chunks loaded: 397


**Inspect a Chunk (Sanity Check)**

In [36]:
# Inspect one chunk

sample = chunks[50]
print("Unit:", sample["unit"])
print("Section:", sample["section_number"], sample["section"])
print("Pages:", sample["page_start"], "-", sample["page_end"])
print("-" * 80)
print(sample["text"][:800])


Unit: UNMAPPED
Section: 1.11 MelTINg of caST IroN
Pages: 69 - 69
--------------------------------------------------------------------------------
In metal casting, melting is considered to be important; because it controls the quality of the casting. A wide range of furnaces such as the pit furnace, open hearth furnace, rotary furnace, cupola furnace, and electric arc furnace are available for melting. The choice of the furnace is made based upon the amount and type of metals or alloys to be melted. A cupola furnace is most commonly used for melting cast iron.


**Load Embedding Model (Frozen Choice)**

*We use a deterministic, lightweight, academic-safe model.*

In [37]:
# Load embedding model (CPU)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


**Prepare Text for Embedding**

In [38]:
# Prepare texts for embedding

texts = [chunk["text"] for chunk in chunks]

**Generate Embeddings**

In [39]:
# Generate embeddings

embeddings = embedding_model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Embeddings shape: (397, 384)


**Build FAISS Index**

*We use cosine similarity, implemented via inner product on normalized vectors.*

In [40]:
# Build FAISS index

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print("Total vectors in FAISS index:", index.ntotal)

Total vectors in FAISS index: 397


**Save Index & Metadata**

In [41]:
# Save FAISS index and metadata

FAISS_INDEX_PATH = "/content/drive/MyDrive/RAG/faiss_index.bin"
METADATA_PATH = "/content/drive/MyDrive/RAG/semantic_chunks.json"

faiss.write_index(index, FAISS_INDEX_PATH)

# Save metadata separately (no embeddings)
with open(METADATA_PATH, "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=2, ensure_ascii=False)

print("FAISS index and metadata saved.")

FAISS index and metadata saved.


**Manual Retrieval Test (NO LLM)**

In [42]:
# Manual similarity search test

def search(query, top_k=5):
    q_emb = embedding_model.encode(
        [query],
        normalize_embeddings=True
    )
    scores, indices = index.search(q_emb, top_k)
    return scores[0], indices[0]


query = "define metal casting process"
scores, ids = search(query)

for score, idx in zip(scores, ids):
    ch = chunks[idx]
    print("="*80)
    print("Score:", round(float(score), 3))
    print("Unit:", ch["unit"])
    print("Section:", ch["section_number"], ch["section"])
    print("Pages:", ch["page_start"], "-", ch["page_end"])
    print(ch["text"][:400])


Score: 0.78
Unit: UNIT III
Section: 1.7 INTrodUcTIoN To MeTal caSTINg proceSS
Pages: 55 - 55
Metal casting is one of the most versatile forms of production processes. There is no limit to the size and shape of the articles that can be produced by casting. The production cost is considerably low. Although all metals can be cast, iron is mostly used because of its fluidity, small shrinkage and ease with which its properties are controlled. Casting process involves the pouring of molten meta
Score: 0.653
Unit: UNMAPPED
Section: 1.16 Introduction To Metal Joining Processes
Pages: 75 - 75
Metal joining processes are manufacturing processes in which metal pieces are joined by the application of heat by some means. The various metal joining processes are welding, brazing and soldering. These processes differ from each other in the temperature at which the joining is done. Welding involves heating the metals over a higher range of temperature up to fusion and then allowing it to cool.
Score: 0

**Similarity Threshold Check**

In [43]:
# Inspect similarity scores

for score in scores:
    print(round(float(score), 3))


0.78
0.653
0.634
0.586
0.586
